In [1]:
import sys
import pandas as pd
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, LLMPredictor, PromptHelper
from llama_index.storage.storage_context import StorageContext
from llama_index.llm_predictor import HuggingFaceLLMPredictor
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from IPython.display import Markdown, display
from llama_index import ServiceContext, LangchainEmbedding
import os
import pandas as pd
from transformers import LlamaTokenizer, LlamaForCausalLM
import torch
from llama_index.prompts.prompts import SimpleInputPrompt
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any
from langchain.llms import OpenAI
import logging
from langchain.chat_models import ChatOpenAI
from llama_index import QuestionAnswerPrompt
import gc


sys.path.insert(0, '..')


topic_df = pd.read_parquet("gs://scraped-news-article-data-null/2023-topics-%s.parquet" % "openai")
topic_df.head()

,topics,summary
0,0,The news articles revolve around the advanceme...
1,1,The news articles cover the fluctuations in oi...
2,2,The news articles revolve around the security ...
3,3,The Federal Reserve's interest rate decisions ...
4,4,The news articles cover various aspects of the...


In [2]:
import sys


logging.basicConfig(level=logging.WARN, stream=sys.stdout)

In [3]:
topic_existing_sum = topic_df.loc[(topic_df.summary.str.len() > 0) & (topic_df.summary.str.lower().str.strip() != "no theme")]
topic_existing_sum.summary.head()

0    The news articles revolve around the advanceme...
1    The news articles cover the fluctuations in oi...
2    The news articles revolve around the security ...
3    The Federal Reserve's interest rate decisions ...
4    The news articles cover various aspects of the...
Name: summary, dtype: object

In [4]:
from summarizer.topic_sum import create_topic_filter, load_faiss_topic_filter

In [ ]:
with open("/home/jupyter/apikey", "r") as api_fp:
    api_key = api_fp.read().strip()


filter_llm = create_topic_filter(api_key=api_key, temperature=0)
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))

In [ ]:
class LlamaBased(LLM):

    tokenizer: LlamaTokenizer
    model: LlamaForCausalLM
    max_new_tokens: int
    query_helper: SimpleInputPrompt
    
    def __init__(self, model_path, max_new_tokens, query_helper):
        tokenizer = LlamaTokenizer.from_pretrained(model_path, device_map="auto", max_input_size=2048)
        model = LlamaForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")
        LLM.__init__(self, tokenizer=tokenizer, model=model, max_new_tokens=max_new_tokens, query_helper=query_helper)
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        prompt = self.query_helper.format(query_str=prompt)
        logging.info("final prompt: " + prompt)
        inputs = self.tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = self.model.generate(**inputs, max_new_tokens=self.max_new_tokens, 
                             do_sample=False, 
                             temperature=0)
        result = self.tokenizer.batch_decode(outputs, skip_special_tokens=True, spaces_between_special_tokens=False)[0]
        # only return newly generated tokens
        return result[len(prompt):]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": "Koala"}

    @property
    def _llm_type(self) -> str:
        return "llama"
    

In [ ]:
def create_llama_service_context(embed_model, model_path, query_wrapper_prompt, num_output=512, max_chunk_overlap=20):
    llm_predictor = LLMPredictor(llm=LlamaBased(model_path, num_output, query_wrapper_prompt))
    prompt_helper = PromptHelper(max_input_size=2048, num_output=num_output, max_chunk_overlap=max_chunk_overlap)
    service_context = ServiceContext.from_defaults(chunk_size_limit=512, llm_predictor=llm_predictor, embed_model=embed_model, prompt_helper=prompt_helper)
    return service_context


def create_openai_service_context(embed_model, api_key, num_output=512, max_chunk_overlap=20):
    prompt_helper = PromptHelper(max_input_size=4096, num_output=num_output, max_chunk_overlap=max_chunk_overlap)
    service_context = ServiceContext.from_defaults(chunk_size_limit=512, 
                                                   embed_model=embed_model, 
                                                   prompt_helper=prompt_helper,
                                                   llm_predictor=LLMPredictor(llm=ChatOpenAI(model_name="gpt-3.5-turbo", 
                                                                                             temperature=0, 
                                                                                             openai_api_key=api_key))
                                                  )
    return service_context
    

In [ ]:
from llama_index import load_index_from_storage, load_indices_from_storage, load_graph_from_storage
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.vector_stores import SimpleVectorStore
from llama_index.storage.index_store import SimpleIndexStore
from pathlib import Path


persist_dir_base = "/home/jupyter/topic_indices"


def create_llama_index_lazyloader(service_context, directory):
    index_dict = {}
    
    def get_llama_index(topic_number):
        if topic_number not in index_dict:
            topic_index_dir = Path(directory, str(topic_number))
            if not topic_index_dir.exists():
                return None
            storage_context = StorageContext.from_defaults(
                docstore=SimpleDocumentStore.from_persist_dir(persist_dir=topic_index_dir),
                vector_store=SimpleVectorStore.from_persist_dir(persist_dir=topic_index_dir),
                index_store=SimpleIndexStore.from_persist_dir(persist_dir=topic_index_dir),
            )
            index = load_index_from_storage(storage_context=storage_context, service_context=service_context)
            index_dict[topic_number] = index
        return index_dict[topic_number]
            
    return get_llama_index
        

In [ ]:
faiss_path = "/home/jupyter/faiss_topic.index"


lazy_loader = create_llama_index_lazyloader(create_openai_service_context(embed_model, api_key), persist_dir_base)
faiss_topic_filter = load_faiss_topic_filter(faiss_path, embed_model=embed_model)

In [10]:
inquiry = "Which companies are developing drugs?"


OPENAI_FINAL_PROMPT = "Several extracts from news articles are provided below. " + \
"Using only information from the news extracts, write a well written response with a heading that answers the given inquiry. " + \
"The response should read well in addition to providing relevant and accurate information." + \
'If the inqury cannot be answered using only the information from the provided article extracts, write "CANNOT ANSWER" as the response.' +\
'The response should be formatted in plain text.\n\n' + \
"Inquiry: {query_str}\n\nNews Article Extracts:\n{context_str}\n"


QA_PROMPT = QuestionAnswerPrompt(OPENAI_FINAL_PROMPT)


topic_filtered = faiss_topic_filter(inquiry, topic_existing_sum)
for r in filter_llm(topic_filtered, inquiry):
    if r.rating < 0.5:
        continue
    index = lazy_loader(r.topic_number)
    if not index:
        continue
    query_engine = index.as_query_engine(text_qa_template=QA_PROMPT)
    result_str = query_engine.query(inquiry)
    logging.info("Generated response: %s" % result_str.response)
#    if "INSUFFICIENT INFORMATION" in result_str.response:
#        continue
    print("----------------------------------------------")
    print(result_str.response)
    print("----------------------------------------------")

----------------------------------------------
CANNOT ANSWER
----------------------------------------------
----------------------------------------------
CANNOT ANSWER
----------------------------------------------
----------------------------------------------
Response: Novo Nordisk is developing drugs for diabetes and obesity. The company has developed the highly popular Wegovy obesity drug, which has been launched in the U.S., Denmark, and Norway. The company expects the drug to be available in more European markets this year. Novo Nordisk also has a best-selling diabetes treatment drug called Ozempic. The company has faced some supply constraints this year, but a second contract manufacturer is ready to begin production of Wegovy, which would increase supply. In the first quarter of this year, Novo Nordisk's sales increased by 25% and operating profit rose by 28% in local currencies. The company will present full financial results for the first quarter of this year on May 4.
-----

In [11]:
inquiry = "Which companies are developing new AI products?"


topic_filtered = faiss_topic_filter(inquiry, topic_existing_sum)
for r in filter_llm(topic_filtered, inquiry):
    if r.rating < 0.5:
        continue
    index = lazy_loader(r.topic_number)
    if not index:
        continue
    query_engine = index.as_query_engine(text_qa_template=QA_PROMPT)
    result_str = query_engine.query(inquiry)
    logging.info("Generated response: %s" % result_str.response)
#    if "INSUFFICIENT INFORMATION" in result_str.response:
#        continue
    print("----------------------------------------------")
    print(result_str.response)
    print("----------------------------------------------")

----------------------------------------------
Companies Developing New AI Products: Alphabet, Microsoft, Amazon, and Meta

According to recent news articles, Alphabet, Microsoft, Amazon, and Meta are all developing new AI products. Google, a subsidiary of Alphabet, recently launched its own chatbot called Bard AI to rival Microsoft and OpenAI's ChatGPT. Tech investors are eager to see how much these industry leaders are investing in AI, as it is an area where they want to see hefty investments. Generative AI programs require specialized supercomputers that are not cheap, but tech CEOs emphasized the large sums of money they will be spending to build and run these applications. Sundar Pichai, CEO of Alphabet, is under intense pressure to deliver AI products due to the perceived threat that Alphabet's core Google search engine faces from the sophisticated chatbots hitting the market. Alphabet recently declared an internal "code red."
----------------------------------------------
------

In [15]:
system = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. "
query_wrapper_prompt = SimpleInputPrompt(system + "USER: {query_str} ASSISTANT:")

In [16]:
del query_engine
gc.collect()
torch.cuda.empty_cache()
lazy_loader = create_llama_index_lazyloader(create_llama_service_context(embed_model, "/home/jupyter/vicuna-7b", query_wrapper_prompt), persist_dir_base)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
VICUNA_FINAL_PROMPT = "Several extracts from news articles are provided below. " + \
"Using only information from the news extracts, answer the given inquiry. " + \
"The answer should read well in addition to providing relevant and accurate information." + \
'If the inqury cannot be answered, write "CANNOT ANSWER" as the response.' +\
'Do not simply repeat the news extracts. ' +\
'The response should be formatted in plain text.\n\n' + \
"Inquiry: {query_str}\n\nNews Article Extracts:\n{context_str}\n"


QA_PROMPT = QuestionAnswerPrompt(VICUNA_FINAL_PROMPT)
inquiry = "Which companies are developing drugs?"


topic_filtered = faiss_topic_filter(inquiry, topic_existing_sum)
for r in filter_llm(topic_filtered, inquiry):
    if r.rating < 0.5:
        continue
    index = lazy_loader(r.topic_number)
    if not index:
        continue
    query_engine = index.as_query_engine(text_qa_template=QA_PROMPT)
    result_str = query_engine.query(inquiry)
    logging.info("Generated response: %s" % result_str.response)
#    if "INSUFFICIENT INFORMATION" in result_str.response:
#        continue
    print("----------------------------------------------")
    print(result_str.response)
    print("----------------------------------------------")

----------------------------------------------
BeiGene Ltd and Swiss firm Novartis AG are developing an experimental drug that could become the first-line of therapy for patients with advanced forms of gastric cancer. The drug is currently approved for 10 different cancer indications in China alone, but is not approved in the United States. The experimental drug, in combination with chemotherapy, prolonged survival in patients whose gastric cancer has either spread to other parts of the body, or is in advanced stages. The drug is currently under review by the U.S. Food and Drug Administration as a second-line therapy for another type of esophageal cancer after prior chemotherapy. No other companies developing drugs are mentioned in the news extracts.
----------------------------------------------
----------------------------------------------
AbbVie Inc is developing drugs, including Skyrizi for psoriasis and related conditions, Rinvoq for several autoimmune disorders, and Botox for co

In [11]:
inquiry = "Which companies are developing new AI products?"


topic_filtered = faiss_topic_filter(inquiry, topic_existing_sum)
for r in filter_llm(topic_filtered, inquiry):
    if r.rating < 0.5:
        continue
    index = lazy_loader(r.topic_number)
    if not index:
        continue
    query_engine = index.as_query_engine(text_qa_template=QA_PROMPT)
    result_str = query_engine.query(inquiry)
    logging.info("Generated response: %s" % result_str.response)
#    if "INSUFFICIENT INFORMATION" in result_str.response:
#        continue
    print("----------------------------------------------")
    print(result_str.response)
    print("----------------------------------------------")

----------------------------------------------
Alphabet, Microsoft, Amazon, and Meta are all developing new AI products, including large language models (LLMs) that power chatbots like Google's Bard AI, which is designed to rival Microsoft and OpenAI's ChatGPT. Tech CEOs are emphasizing the need for large investments in AI, including specialized supercomputers for generative AI programs that produce outputs that seem like they were made by a human. Sundar Pichai, CEO of Alphabet, is under pressure to deliver AI products due to the perceived threat to Google's search engine from sophisticated chatbots.
----------------------------------------------
----------------------------------------------
Qualcomm is developing a chip called the Cloud AI 100 that aims for parsimonious power consumption. In testing data published on Wednesday by MLCommons, Qualcomm's AI 100 beat Nvidia's flagship H100 chip at classifying images and object detection. Nvidia, however, took the top spot in both absolu